In [4]:
import argparse
import os
import ee
import ee.mapclient
import fiona
import rasterio
from shapely.geometry import Polygon, LineString
from skimage import measure, draw
from shapely import ops
import pandas
import numpy as np
import geemap as geemap
import shutil
from natsort import natsorted
import folium
from folium import plugins

In [2]:
ee.Initialize()

In [5]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [6]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [31]:
polygon_path = '/Users/Evan/Documents/Mobility/GIS/Taiwan/Taiwan1/Taiwan1.gpkg'
polygon_name = polygon_path.split('/')[-1].split('.')[0]
with fiona.open(polygon_path, layer=polygon_name) as layer:
    for feature in layer:
        geom = feature['geometry']
        poly = ee.Geometry.Polygon(geom['coordinates'])

In [48]:
#ds = ee.Image('WWF/HydroSHEDS/03CONDEM')
ds = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_12").filterBounds(
    poly
)

dataset_vis = {
  'color': "B2B2B3",
  'width': 1.0,
}

In [92]:
basin_ids = []
for feature in ds.getInfo()['features']:
    basin_id = feature['properties']['HYBAS_ID']
    basin_ids.append(basin_id)

print(basin_ids)
upstreams = basin_ids

[4120969680, 4120972530, 4120972420]


In [93]:
sheds = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_12")

while len(new_upstreams):
    new_upstreams = []
    for upstream in upstreams:
        shed_filter = sheds.filter(ee.Filter.eq('NEXT_DOWN', upstream))
        upstreams = []
        for feature in shed_filter.getInfo()['features']:
            new_upstreams.append(feature['properties']['HYBAS_ID'])

    upstreams = new_upstreams
    basin_ids += new_upstreams
    
print(upstreams)
print(basin_ids)

[]
[4120969680, 4120972530, 4120972420, 4120972530, 4120972420, 4120974340, 4120974380, 4121522810, 4120974340, 4120974380, 4121522810, 4120974820, 4120975050, 4121526670, 4120974820, 4120975050, 4121526670, 4120972240, 4120972310, 4120972240, 4120972310]


In [107]:
shed_polys = []
for basin_id in basin_ids:
    shed = sheds.filter(ee.Filter.eq('HYBAS_ID', basin_id)).getInfo()['features'][0]['geometry']
    shed_polys.append(ee.Feature(ee.Geometry.Polygon(shed['coordinates'])))

fcShed = ee.FeatureCollection(shed_polys)
area_shed = fcShed.union(1).geometry()

In [111]:
type(poly)

ee.geometry.Geometry

In [108]:
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[23.35, 121.33], zoom_start=10, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(poly, {}, 'poly')
my_map.add_ee_layer(area_shed, dataset_vis, '')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)